In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import sys
print(sys.version)
sys.path.append('../')

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]


In [2]:
# import slip dataset

dataset_path = '../data/ral2023_dataset/husky/boreal_snow/slip_dataset_all.pkl'

full_dataset = pd.read_pickle(dataset_path)
full_dataset_length = len(full_dataset)
train_dataset = full_dataset[:int(full_dataset_length/2)]
validate_dataset = full_dataset[int(full_dataset_length/2):]

print(train_dataset)

     init_icp_x  init_icp_y  init_icp_z  init_icp_roll  init_icp_pitch  \
0           0.0         0.0         0.0            0.0             0.0   
1           0.0         0.0         0.0            0.0             0.0   
2           0.0         0.0         0.0            0.0             0.0   
3           0.0         0.0         0.0            0.0             0.0   
4           0.0         0.0         0.0            0.0             0.0   
..          ...         ...         ...            ...             ...   
296         0.0         0.0         0.0            0.0             0.0   
297         0.0         0.0         0.0            0.0             0.0   
298         0.0         0.0         0.0            0.0             0.0   
299         0.0         0.0         0.0            0.0             0.0   
300         0.0         0.0         0.0            0.0             0.0   

     init_icp_yaw  calib_step  cmd_left_0  cmd_right_0  cmd_left_1  ...  \
0             0.0         0.0    6.5

In [ ]:
# compute M-RMSE

def compute_single_step_error(meas_state, pred_state):
    pred_error = meas_state - pred_state
    pred_error[2] = wrap2pi(pred_error[2]) # wrap yaw angle error to pi
    pred_error_squared_scalar = pred_error.T @ pred_error
    return pred_error_squared_scalar

def horizon_mrsme(meas_states, pred_states):
    n_predictions = meas_states.shape[0]
    prediction_error_sum = 0
    for i in range(0, n_predictions):
        prediction_error_sum += compute_single_step_error(meas_states[i], pred_states[i])
    return np.sqrt(prediction_error_sum / n_predictions)

n_test_windows = 300

blr_predictions = np.zeros((n_test_windows, window_size, 3))
idd_predictions = np.zeros((n_test_windows, window_size, 3))

for j in range(0, n_test_windows):
    x_horizon_x = test_idd_body_vel_x_array[j, :]
    x_horizon_y = test_idd_body_vel_x_array[j, :] * test_idd_body_vel_yaw_array[j, :]
    x_horizon_yaw = np.column_stack((test_idd_body_vel_x_array[j, :] * test_idd_body_vel_yaw_array[j, :], 
                                   test_idd_body_vel_x_array[j, :], 
                                   test_idd_body_vel_yaw_array[j, :]))

    horizon_blr_slip_x_mean, horizon_blr_slip_x_variance = slip_blr_x.predict_slip(x_horizon_x)
    horizon_blr_slip_y_mean, horizon_blr_slip_y_variance = slip_blr_y.predict_slip(x_horizon_y)
    horizon_blr_slip_yaw_mean, horizon_blr_slip_yaw_variance = slip_blr_yaw.predict_slip(x_horizon_yaw)
    for i in range(1, window_size):
        blr_body_vel = np.array([test_idd_body_vel_x_array[j, i-1] - horizon_blr_slip_x_mean[i-1], test_idd_body_vel_y_array[j, i-1] - horizon_blr_slip_y_mean[i-1]]).reshape(2,1)
        # print(blr_body_vel)
        yaw_to_rotmat2d(blr_body_to_world_rotmat, blr_slip_states[i-1, 2])
        blr_world_vel = blr_body_to_world_rotmat @ blr_body_vel

        blr_predictions[j, i, 0] = blr_predictions[j, i-1, 0] + blr_world_vel[0] * dt
        blr_predictions[j, i, 1] = blr_predictions[j, i-1, 1] + blr_world_vel[1] * dt
        blr_predictions[j, i, 2] = blr_predictions[j, i-1, 2] + (test_idd_body_vel_yaw_array[j, i-1] - horizon_blr_slip_yaw_mean[i-1]) * dt
        
        idd_predictions[j, i, 0] = idd_predictions[j, i-1, 0] + np.cos(idd_predictions[j, i-1, 2]) * test_idd_body_vel_x_array[j, i-1] * dt
        idd_predictions[j, i, 1] = idd_predictions[j, i-1, 1] + np.sin(idd_predictions[j, i-1, 2]) * test_idd_body_vel_x_array[j, i-1] * dt
        idd_predictions[j, i, 2] = idd_predictions[j, i-1, 2] + test_idd_body_vel_yaw_array[j, i-1] * dt
    